In [1]:
%env OMP_NUM_THREADS = 1

In [2]:
import torch
import os

from ocpmodels.common import logger
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import setup_logging

from cdm.charge_trainer import ChargeTrainer
from cdm import models

setup_logging()

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
#torch.cuda.is_available = lambda: False

if torch.cuda.is_available():
    print("True")
else:
    print("False")
    torch.set_num_threads(8)

True


In [4]:
task = {
    'description': 'Predicting electron density from atomic positions',
    'dataset': 'lmdb',
}

In [13]:
model = {
    'name': 'charge_model',
    'enforce_zero_for_disconnected_probes': True,
    'enforce_charge_conservation': True,
    'freeze_atomic': True,
    
    'atom_model_config': {
        'name': 'schnet_charge',
        'checkpoint': '/home/jovyan/shared-scratch/ocp_checkpoints/public_checkpoints/schnet_all_large.pt'
    },
    
    'probe_model_config': {
        'name': 'schnet_charge',
        'num_interactions': 3,
        'cutoff': 5,
    },
    
    'otf_pga_config': {
        'num_probes': 100000,
        'cutoff': 6,
    }
}

In [14]:
optimizer = {
    'optimizer': 'Adam',
    'num_workers': 4,
    'lr_initial': 5e-5,
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.96,
    'patience': 1,
    'max_epochs': 1000,
    'loss_charge': 'normed_mae'
}

In [15]:
tg

In [16]:
trainer_config = {
    'trainer': 'charge',
    'identifier': 'SchNet cp fz + SchNet',
    'is_debug': True,
    'run_dir': '../runs/',
    'print_every': 1,
    'seed': 2,
    'logger': 'wandb',
    'local_rank': 0,
    'amp': True,
}

In [17]:
trainer = registry.get_trainer_class(
    trainer_config['trainer'])(task = task,
                               model = model,
                               dataset = dataset,
                               optimizer = optimizer,
                               **trainer_config)

amp: true
cmd:
  checkpoint_dir: ../runs/checkpoints/2022-12-16-17-59-28-SchNet cp fz + SchNet
  commit: 7d1040b
  identifier: SchNet cp fz + SchNet
  logs_dir: ../runs/logs/wandb/2022-12-16-17-59-28-SchNet cp fz + SchNet
  print_every: 1
  results_dir: ../runs/results/2022-12-16-17-59-28-SchNet cp fz + SchNet
  seed: 2
  timestamp_id: 2022-12-16-17-59-28-SchNet cp fz + SchNet
dataset:
  src: ../../charge-data/1k-no-probe-graphs/train
gpus: 1
logger: wandb
model: charge_model
model_attributes:
  atom_model_config:
    checkpoint: /home/jovyan/shared-scratch/ocp_checkpoints/public_checkpoints/schnet_all_large.pt
    name: schnet_charge
  enforce_charge_conservation: true
  enforce_zero_for_disconnected_probes: true
  freeze_atomic: true
  otf_pga_config:
    cutoff: 6
    num_probes: 100000
  probe_model_config:
    cutoff: 5
    name: schnet_charge
    num_interactions: 3
noddp: false
optim:
  factor: 0.96
  loss_charge: normed_mae
  lr_initial: 5.0e-05
  max_epochs: 1000
  mode: min
 

In [18]:
trainer.model.module

ChargeModel(
  (atom_message_model): schnet_charge(hidden_channels=1024, num_filters=256, num_interactions=5, num_gaussians=200, cutoff=6.0)
  (probe_message_model): schnet_charge(hidden_channels=128, num_filters=128, num_interactions=3, num_gaussians=50, cutoff=5)
  (atom_reduction): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=128, bias=True)
  )
  (probe_output_function): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:
trainer.train()

/home/jovyan/charge-density-models/cdm/utils/probe_graph.py:188: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probe_data.target = torch.tensor(density.reshape(density.shape[-3:])[probe_choice[0:3]], device = probe_pos.device)


norm_charge_mae: 1.27e+00, norm_charge_rmse: 6.82e-01, charge_mse: 1.74e-02, charge_mae: 4.75e-02, true_density: 3.73e-02, total_charge_ratio: 9.87e-01, loss: 2.55e+00, lr: 5.00e-05, epoch: 1.11e-03, step: 1.00e+00
norm_charge_mae: 1.15e+00, norm_charge_rmse: 6.50e-01, charge_mse: 2.70e-02, charge_mae: 7.37e-02, true_density: 6.39e-02, total_charge_ratio: 1.00e+00, loss: 1.15e+00, lr: 5.00e-05, epoch: 2.22e-03, step: 2.00e+00
norm_charge_mae: 1.20e+00, norm_charge_rmse: 1.77e+00, charge_mse: 6.72e-01, charge_mae: 2.57e-01, true_density: 2.15e-01, total_charge_ratio: 9.95e-01, loss: 2.39e+00, lr: 5.00e-05, epoch: 3.33e-03, step: 3.00e+00
norm_charge_mae: 1.01e+00, norm_charge_rmse: 8.01e-01, charge_mse: 4.35e-02, charge_mae: 6.85e-02, true_density: 6.77e-02, total_charge_ratio: 1.01e+00, loss: 2.02e+00, lr: 5.00e-05, epoch: 4.44e-03, step: 4.00e+00
norm_charge_mae: 1.07e+00, norm_charge_rmse: 1.13e+00, charge_mse: 2.09e-01, charge_mae: 1.77e-01, true_density: 1.65e-01, total_charge_rati

norm_charge_mae: 8.11e-01, norm_charge_rmse: 1.11e+00, charge_mse: 1.74e-01, charge_mae: 1.15e-01, true_density: 1.42e-01, total_charge_ratio: 9.95e-01, loss: 8.11e-01, lr: 5.00e-05, epoch: 4.44e-02, step: 4.00e+01
norm_charge_mae: 8.38e-01, norm_charge_rmse: 1.26e+00, charge_mse: 1.20e-01, charge_mae: 6.37e-02, true_density: 7.60e-02, total_charge_ratio: 1.02e+00, loss: 8.38e-01, lr: 5.00e-05, epoch: 4.56e-02, step: 4.10e+01
norm_charge_mae: 1.09e+00, norm_charge_rmse: 7.15e-01, charge_mse: 1.38e-02, charge_mae: 2.96e-02, true_density: 2.71e-02, total_charge_ratio: 1.03e+00, loss: 2.19e+00, lr: 5.00e-05, epoch: 4.67e-02, step: 4.20e+01
norm_charge_mae: 9.90e-01, norm_charge_rmse: 1.78e+00, charge_mse: 8.06e-01, charge_mae: 2.53e-01, true_density: 2.55e-01, total_charge_ratio: 9.92e-01, loss: 9.90e-01, lr: 5.00e-05, epoch: 4.78e-02, step: 4.30e+01
norm_charge_mae: 7.34e-01, norm_charge_rmse: 4.12e-01, charge_mse: 7.93e-03, charge_mae: 3.43e-02, true_density: 4.67e-02, total_charge_rati

norm_charge_mae: 6.86e-01, norm_charge_rmse: 5.14e-01, charge_mse: 1.31e-02, charge_mae: 3.41e-02, true_density: 4.96e-02, total_charge_ratio: 1.01e+00, loss: 6.86e-01, lr: 5.00e-05, epoch: 8.78e-02, step: 7.90e+01
norm_charge_mae: 9.00e-01, norm_charge_rmse: 1.02e+00, charge_mse: 1.37e-01, charge_mae: 1.18e-01, true_density: 1.31e-01, total_charge_ratio: 1.01e+00, loss: 9.00e-01, lr: 5.00e-05, epoch: 8.89e-02, step: 8.00e+01
norm_charge_mae: 9.16e-01, norm_charge_rmse: 1.41e+00, charge_mse: 4.09e-01, charge_mae: 1.89e-01, true_density: 2.07e-01, total_charge_ratio: 1.01e+00, loss: 9.16e-01, lr: 5.00e-05, epoch: 9.00e-02, step: 8.10e+01
norm_charge_mae: 5.98e-01, norm_charge_rmse: 3.75e-01, charge_mse: 7.97e-03, charge_mae: 3.39e-02, true_density: 5.65e-02, total_charge_ratio: 9.94e-01, loss: 5.98e-01, lr: 5.00e-05, epoch: 9.11e-02, step: 8.20e+01
norm_charge_mae: 5.66e-01, norm_charge_rmse: 7.63e-01, charge_mse: 1.07e-01, charge_mae: 1.04e-01, true_density: 1.84e-01, total_charge_rati

norm_charge_mae: 6.62e-01, norm_charge_rmse: 6.54e-01, charge_mse: 3.70e-02, charge_mae: 5.73e-02, true_density: 8.62e-02, total_charge_ratio: 1.01e+00, loss: 6.62e-01, lr: 5.00e-05, epoch: 1.31e-01, step: 1.18e+02
norm_charge_mae: 9.01e-01, norm_charge_rmse: 1.11e+00, charge_mse: 1.20e-01, charge_mae: 8.79e-02, true_density: 9.73e-02, total_charge_ratio: 9.96e-01, loss: 9.01e-01, lr: 5.00e-05, epoch: 1.32e-01, step: 1.19e+02
norm_charge_mae: 8.45e-01, norm_charge_rmse: 1.65e+00, charge_mse: 4.92e-01, charge_mae: 1.52e-01, true_density: 1.80e-01, total_charge_ratio: 1.01e+00, loss: 8.45e-01, lr: 5.00e-05, epoch: 1.33e-01, step: 1.20e+02
norm_charge_mae: 6.07e-01, norm_charge_rmse: 4.37e-01, charge_mse: 1.02e-02, charge_mae: 3.24e-02, true_density: 5.33e-02, total_charge_ratio: 9.87e-01, loss: 6.07e-01, lr: 5.00e-05, epoch: 1.34e-01, step: 1.21e+02
norm_charge_mae: 8.16e-01, norm_charge_rmse: 1.09e+00, charge_mse: 1.23e-01, charge_mae: 8.40e-02, true_density: 1.03e-01, total_charge_rati

...

norm_charge_mae: 7.17e-01, norm_charge_rmse: 1.08e+00, charge_mse: 1.84e-01, charge_mae: 1.13e-01, true_density: 1.57e-01, total_charge_ratio: 9.95e-01, loss: 7.17e-01, lr: 5.00e-05, epoch: 4.34e-01, step: 3.91e+02
norm_charge_mae: 6.08e-01, norm_charge_rmse: 5.50e-01, charge_mse: 1.88e-02, charge_mae: 3.79e-02, true_density: 6.21e-02, total_charge_ratio: 1.00e+00, loss: 6.08e-01, lr: 5.00e-05, epoch: 4.36e-01, step: 3.92e+02
norm_charge_mae: 4.83e-01, norm_charge_rmse: 4.05e-01, charge_mse: 2.07e-02, charge_mae: 6.07e-02, true_density: 1.26e-01, total_charge_ratio: 1.00e+00, loss: 4.83e-01, lr: 5.00e-05, epoch: 4.37e-01, step: 3.93e+02
norm_charge_mae: 6.42e-01, norm_charge_rmse: 8.76e-01, charge_mse: 1.27e-01, charge_mae: 1.07e-01, true_density: 1.66e-01, total_charge_ratio: 1.01e+00, loss: 6.42e-01, lr: 5.00e-05, epoch: 4.38e-01, step: 3.94e+02
norm_charge_mae: 7.42e-01, norm_charge_rmse: 1.28e+00, charge_mse: 1.54e-01, charge_mae: 6.92e-02, true_density: 9.32e-02, total_charge_rati

norm_charge_mae: 8.34e-01, norm_charge_rmse: 8.64e-01, charge_mse: 8.06e-02, charge_mae: 9.01e-02, true_density: 1.08e-01, total_charge_ratio: 9.97e-01, loss: 8.34e-01, lr: 5.00e-05, epoch: 4.78e-01, step: 4.30e+02
norm_charge_mae: 9.93e-01, norm_charge_rmse: 1.64e+00, charge_mse: 6.09e-01, charge_mae: 2.26e-01, true_density: 2.28e-01, total_charge_ratio: 1.00e+00, loss: 9.93e-01, lr: 5.00e-05, epoch: 4.79e-01, step: 4.31e+02
norm_charge_mae: 7.19e-01, norm_charge_rmse: 8.88e-01, charge_mse: 1.44e-01, charge_mae: 1.31e-01, true_density: 1.83e-01, total_charge_ratio: 1.01e+00, loss: 7.19e-01, lr: 5.00e-05, epoch: 4.80e-01, step: 4.32e+02
norm_charge_mae: 7.17e-01, norm_charge_rmse: 4.31e-01, charge_mse: 1.11e-02, charge_mae: 4.27e-02, true_density: 5.95e-02, total_charge_ratio: 1.00e+00, loss: 7.17e-01, lr: 5.00e-05, epoch: 4.81e-01, step: 4.33e+02
norm_charge_mae: 9.57e-01, norm_charge_rmse: 1.77e+00, charge_mse: 6.07e-01, charge_mae: 1.85e-01, true_density: 1.94e-01, total_charge_rati

norm_charge_mae: 5.24e-01, norm_charge_rmse: 6.27e-01, charge_mse: 7.18e-02, charge_mae: 9.57e-02, true_density: 1.82e-01, total_charge_ratio: 9.87e-01, loss: 5.24e-01, lr: 5.00e-05, epoch: 5.21e-01, step: 4.69e+02
norm_charge_mae: 6.18e-01, norm_charge_rmse: 9.02e-01, charge_mse: 1.55e-01, charge_mae: 1.18e-01, true_density: 1.90e-01, total_charge_ratio: 9.99e-01, loss: 6.18e-01, lr: 5.00e-05, epoch: 5.22e-01, step: 4.70e+02
norm_charge_mae: 6.51e-01, norm_charge_rmse: 6.13e-01, charge_mse: 1.97e-02, charge_mae: 3.42e-02, true_density: 5.23e-02, total_charge_ratio: 9.97e-01, loss: 6.51e-01, lr: 5.00e-05, epoch: 5.23e-01, step: 4.71e+02
norm_charge_mae: 6.88e-01, norm_charge_rmse: 5.04e-01, charge_mse: 1.46e-02, charge_mae: 3.95e-02, true_density: 5.73e-02, total_charge_ratio: 9.95e-01, loss: 6.88e-01, lr: 5.00e-05, epoch: 5.24e-01, step: 4.72e+02
norm_charge_mae: 6.79e-01, norm_charge_rmse: 5.90e-01, charge_mse: 2.42e-02, charge_mae: 4.72e-02, true_density: 6.94e-02, total_charge_rati

norm_charge_mae: 7.32e-01, norm_charge_rmse: 1.03e+00, charge_mse: 2.06e-01, charge_mae: 1.42e-01, true_density: 1.94e-01, total_charge_ratio: 1.00e+00, loss: 7.32e-01, lr: 5.00e-05, epoch: 5.64e-01, step: 5.08e+02
norm_charge_mae: 8.13e-01, norm_charge_rmse: 1.35e+00, charge_mse: 1.43e-01, charge_mae: 6.40e-02, true_density: 7.86e-02, total_charge_ratio: 1.00e+00, loss: 8.13e-01, lr: 5.00e-05, epoch: 5.66e-01, step: 5.09e+02
norm_charge_mae: 6.47e-01, norm_charge_rmse: 7.91e-01, charge_mse: 4.42e-02, charge_mae: 4.57e-02, true_density: 7.05e-02, total_charge_ratio: 9.77e-01, loss: 6.47e-01, lr: 5.00e-05, epoch: 5.67e-01, step: 5.10e+02
norm_charge_mae: 5.52e-01, norm_charge_rmse: 4.27e-01, charge_mse: 9.76e-03, charge_mae: 2.96e-02, true_density: 5.30e-02, total_charge_ratio: 9.99e-01, loss: 5.52e-01, lr: 5.00e-05, epoch: 5.68e-01, step: 5.11e+02
norm_charge_mae: 6.97e-01, norm_charge_rmse: 8.86e-01, charge_mse: 8.76e-02, charge_mae: 7.77e-02, true_density: 1.12e-01, total_charge_rati

norm_charge_mae: 4.84e-01, norm_charge_rmse: 4.46e-01, charge_mse: 2.00e-02, charge_mae: 4.86e-02, true_density: 1.00e-01, total_charge_ratio: 1.00e+00, loss: 4.84e-01, lr: 5.00e-05, epoch: 6.08e-01, step: 5.47e+02
norm_charge_mae: 6.35e-01, norm_charge_rmse: 9.47e-01, charge_mse: 1.66e-01, charge_mae: 1.17e-01, true_density: 1.85e-01, total_charge_ratio: 1.00e+00, loss: 6.35e-01, lr: 5.00e-05, epoch: 6.09e-01, step: 5.48e+02
norm_charge_mae: 6.58e-01, norm_charge_rmse: 6.21e-01, charge_mse: 3.06e-02, charge_mae: 5.22e-02, true_density: 7.93e-02, total_charge_ratio: 1.02e+00, loss: 6.58e-01, lr: 5.00e-05, epoch: 6.10e-01, step: 5.49e+02
norm_charge_mae: 8.33e-01, norm_charge_rmse: 9.44e-01, charge_mse: 4.82e-02, charge_mae: 4.51e-02, true_density: 5.42e-02, total_charge_ratio: 1.00e+00, loss: 8.33e-01, lr: 5.00e-05, epoch: 6.11e-01, step: 5.50e+02
norm_charge_mae: 7.07e-01, norm_charge_rmse: 1.06e+00, charge_mse: 1.18e-01, charge_mae: 7.42e-02, true_density: 1.05e-01, total_charge_rati